# Introduction

Copyright

Distribution

In order to use the code the following libraries must be installed:
1. Pandas

In [21]:
import pandas as pd
import numpy as np
from dwave.system.samplers import DWaveSampler
from dwave.system import LeapHybridCQMSampler
from dwave.system.composites import EmbeddingComposite
from dimod import quicksum
from dimod import Binary
from dimod import CQM
import dwave.inspector
import dwavebinarycsp
import re


# Input and Preprocessing (Anna Ehrenberg)

In this section all data input is entered, validated and preprocessed in order to meet further requirements. 

In this section the following input is proceeded:
1. Input data file with information on ferry routes with
    - From: route start port
    - To: route end port
    - Distance: shortest direct distance between departure and arrival port

### Input data file

In [22]:
df_ports = pd.read_csv("sample_data_harbours_utf8.csv", sep = ";")
df_ports

,Column1,Bremerhaven,Brunsbüttel,Emden,Hamburg,Kiel
0,Bremerhaven,0,NaN,137.0,50.0,NaN
1,Brunsbüttel,81,0.0,NaN,36.0,54.0
2,Emden,137,NaN,0.0,NaN,NaN
3,Hamburg,117,36.0,NaN,0.0,90.0
4,Kiel,135,NaN,NaN,NaN,0.0


### Formatting of connection data

In order to formulate the problem in a format accessible by the D-Wave libraries the matrix data is restructured to contain the information in an unstacked format. 

In [23]:
# rename 'Column1' to 'From'
df_ports=df_ports.rename(columns={'Column1':"From"})

# transform end harbours in columns to rows
df_routes= pd.melt(df_ports,id_vars=['From'],var_name="To",value_name='Distance')

# filter NaN-values
df_routes=df_routes[df_routes["Distance"].notnull()]

# filter out 0.0 values
df_routes=df_routes[df_routes.Distance != 0]

#reset index
df_routes=df_routes.reset_index(drop=True)
df_routes_withoutStardEnd=df_routes
df_routes

,From,To,Distance
0,Brunsbüttel,Bremerhaven,81.0
1,Emden,Bremerhaven,137.0
2,Hamburg,Bremerhaven,117.0
3,Kiel,Bremerhaven,135.0
4,Hamburg,Brunsbüttel,36.0
5,Bremerhaven,Emden,137.0
6,Bremerhaven,Hamburg,50.0
7,Brunsbüttel,Hamburg,36.0
8,Brunsbüttel,Kiel,54.0
9,Hamburg,Kiel,90.0


### User input and validation of departure and destination harbour 

In this section the manual input is requested and validated by the following criteria:
1. departure and destination harbour are part of the input 

If input is invalid, the according error message is displayed.

In [24]:
#Check whether the departure harbour is element of the input data
while True:
    #Entering departure harbour
    departure=input(str("Please enter departure port:"))
    if not departure in df_routes['From'].values:
        print("Entered port is not defined as a depature port in the input file. Please enter valid departure port.")
        continue
    else:
        break

#Check whether the destination harbour is element of the input data
while True:
    #Entering destination harbour
    destination=input(str("Please enter destination port:"))
    if not destination in df_routes['To'].values:
        print("Entered port is not defined as a destination port in the input file. Please enter destination valid port.")
        continue
    if destination == departure:
        print("Entered destination port is equal to departure port. Please enter valid destination port.")
        continue
    else:
        #we're happy with the value given.
        #we're ready to exit the loop.
        break


### Data enhancement of connection data

A problem can be formulated in different ways in order to provide the quantum computer with it. In this example, the problem is formulated as a binary quadratic model (BQM) in the form of a QUBO. The mathematical phrase can be attained by a support class called Constraint Satisfaction Problem (CSP) or can directly be input. In this case, the problem will first be represented by a CSP. The CSP constitutes of 
1. all variables and 
2. constraints on the variables.  


# Data Preparation for Quantum Computing (Definition of Classes) (Anna Ehrenberg, Nick Stuke)

In this section, all methods are defined that are later used to
1. create Contraint Satisfaction Problem (CSP)
2. formulate Binary Quadratic Model (BQM) as QUBO. 

### Basic functions

In [25]:
def sum_to_two_or_zero(*args):
        """Checks to see if the args sum to either 0 or 2.
        """
        sum_value = sum(args)
        return sum_value in [0, 2]
        
def sum_smaller_equal_one(*args):
        """Checks to see if the args sum to either smaller or equal to 1.
        """
        sum_value = sum(args)
        return sum_value in [0, 1]

def get_labels(dataframe):
        """Returns a list of labels from a Dataframe of the format of the input file"""
        labels=(dataframe['From']+'-'+dataframe['To']+'-'+dataframe['Distance'].astype(str)).values.tolist()
        return labels

### Definition class Ferryarea

The class Ferryarea is defined as the space in which the routes connect the ports with the given routes.
Each Ferryarea has a start and end point as well as ports and routes between those ports.

As the variables in this example will represent the routes between two ports a set of support variables is needed in order to represent the entered departure and destination port. Therefor, artificial routes from "start" port to each real port and to "end" port from each real port are created in the following code.

In [26]:
class Ferryarea:
    def __init__(self,start,end,routes):
        """ Initializes object of Type Ferryarea"""
        #Instantiate
        self.start=start
        self.end=end
        self.df_routes = routes
        self.ports = pd.DataFrame({'ports':np.append(df_routes['To'].unique(),df_routes['From'].unique())}).drop_duplicates().values.tolist()
        self.csp = dwavebinarycsp.ConstraintSatisfactionProblem(dwavebinarycsp.BINARY)
        self.cqm = CQM()
        self.objectiveDistances=0
        #Add start and end route
        self.df_routes= self.df_routes.append({'From':self.end, 'To':'end','Distance':0}, ignore_index=True)
        self.df_routes= self.df_routes.append({'From':'start', 'To':self.start,'Distance':0}, ignore_index=True)
        
    def _get_csp(self):
        return self.csp
    def _get_cqm(self):
        return self.cqm    
    def _apply_valid_routes_constraint(self):
        """Creates constraints defining that every entered port has to be exited as well and that every port can only be entered and exited through one route."""

        for port in self.ports:
            #assign to directions Dataframe a Dataframe containing only direction to and from the port
            df_directions_from= self.df_routes[self.df_routes['From']==port[0]]
            df_directions_to= self.df_routes[self.df_routes['To']==port[0]]
            df_directions= pd.concat([df_directions_to,df_directions_from])
            
            #creates sets of labels as basis for constraint defintion
            directions_all = set(get_labels(df_directions))
            
            #set((df_directions['From']+'-'+df_directions['To']+'-'+df_directions['Distance'].astype(str)).values.tolist())
            directions_to= set(get_labels(df_directions_to))
            
            #set((df_directions_to['From']+'-'+df_directions_to['To']+'-'+df_directions_to['Distance'].astype(str)).values.tolist())
            directions_from= set(get_labels(df_directions_from))
            #set((df_directions_from['From']+'-'+df_directions_from['To']+'-'+df_directions_from['Distance'].astype(str)).values.tolist())
              
            x={r: Binary(
                "%r" %r
            )for r in directions_all}

            for d in directions_all:
                distance=float(d.split('-')[2])
                self.objectiveDistances=self.objectiveDistances+distance*x[d]
            
            self.cqm.add_constraint(
                quicksum([x[tod] for tod in directions_to])
                -
                quicksum([x[fromd] for fromd in directions_from]) 
                == 0)
            
            #add constraint sum_to_two_or_zero
            self.csp.add_constraint(sum_to_two_or_zero,directions_all)

            #add constraint 'To' sum_to_zero_or_one
            self.csp.add_constraint(sum_smaller_equal_one,directions_to)
            
            #add constraint 'From' sum_to_zero_or_one
            self.csp.add_constraint(sum_smaller_equal_one, directions_from)

    def _set_CQM_objective(self):
        self.cqm.set_objective(self.objectiveDistances)
        
    def _set_start_and_end(self):
        """Sets the values of the departure and destination port to 1 and all other start and end possibilities to 0
        """ #all other possibilies do not have to be 0. However, this is expected to descrease calculation time

        #select all start and end routes
        df_routes_to_fix= pd.concat([self.df_routes[self.df_routes['From']=='start'],self.df_routes[self.df_routes['To']=='end']])
        count=0

        for i in df_routes_to_fix.values:
            label= get_labels(df_routes_to_fix.iloc[[count]])
            if (df_routes_to_fix['To'].iloc[count])==self.start:
                #sets departure to 1
                self.csp.fix_variable(label[0],1)
            elif (df_routes_to_fix['From'].iloc[count])==self.end:
                #sets destination to 1
                self.csp.fix_variable(label[0],1)
            else:
                #sets departure and destination to 0
                self.csp.fix_variable(label[0],0)
            count=count+1

        self.cqm.fix_variable("'start-"+departure+"-0.0'",1)
        self.cqm.fix_variable("'"+destination+"-end-0.0'",1)
        #set start
            #self.csp.fix_variable('start-'+self.start+'-0.0',1)

        #set end
            #self.csp.fix_variable(self.end+'-end-0.0',1)

        #set all other starts and ends to 0
       


### Initialization of Ferryarea

In [27]:
f=Ferryarea(start=departure,end=destination,routes=df_routes)
f._apply_valid_routes_constraint()
f._set_start_and_end()
f._set_CQM_objective()
csp= f._get_csp()
cqm=f._get_cqm()

/tmp/ipykernel_238/1646853718.py:13: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  self.df_routes= self.df_routes.append({'From':self.end, 'To':'end','Distance':0}, ignore_index=True)
/tmp/ipykernel_238/1646853718.py:14: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  self.df_routes= self.df_routes.append({'From':'start', 'To':self.start,'Distance':0}, ignore_index=True)


# Creation of BQM and Transfer to Quantum Sampler (Anna Ehrenberg, Nick Stuke)

In the following Code the CSP is tranformed into a Binary Quadratic Modell formulated in the form of a QUBO. In order to fit the Problem to the modell it is necessary to adjust the following parameters:
- max_graph_size: This parameters limits the number of variables that can be inlcuded in one constraint. In this case, the number of connections to and from one port. The original d-wave function dwavebinarycsp.stitch uses the function np.unpackbits from the numpy library. This only unpacks arrays from the size of 8 bits. Thus, the function all_possible() and the dtype of the matrix A in the file generation.py have been modified. Now, up to 32 connections between variables can be entered. 

In the future, this parameter should be defined by a variable referring to the maximum number of ports. Furthermore, the maximum connections of one port is limited to 32. This should be added as an input validation.  

In [28]:
# Create BQM, max_graph_size gets bigger once more variables are included in one constraint
bqm = dwavebinarycsp.stitch(csp,max_graph_size=24)

In [29]:
# Add Penalty
penalty=0.01
for v in bqm.variables:
    # Ignore auxiliary variables
    if isinstance(v, str) and re.match(r'^aux\d+$', v):
        continue
    split_v = v.split('-')
    bqm.add_variable(v, penalty*float(split_v[2]))

In [30]:
# define Sampler
sampler = EmbeddingComposite(DWaveSampler())


In [31]:
# run BQM on QPU
result = sampler.sample(bqm,
                        num_reads=100,
                        chain_strength=3,
                        label='ferrylines')

In [32]:
# visualize result
dwave.inspector.show(result)

'http://127.0.0.1:18000/?problemId=0e268b16-86c4-46cc-b175-f2d05ef0b296'

In [33]:
print(result.first.sample)

{'Bremerhaven-Emden-137.0': 0, 'Bremerhaven-Hamburg-50.0': 1, 'Brunsbüttel-Bremerhaven-81.0': 0, 'Brunsbüttel-Hamburg-36.0': 0, 'Brunsbüttel-Kiel-54.0': 0, 'Emden-Bremerhaven-137.0': 0, 'Hamburg-Bremerhaven-117.0': 0, 'Hamburg-Brunsbüttel-36.0': 1, 'Hamburg-Kiel-90.0': 0, 'Kiel-Bremerhaven-135.0': 0, 'aux0': 0, 'aux1': 1, 'aux2': 1, 'aux3': 1, 'aux4': 0, 'aux5': 0}


## CQM-Sampling and Result-Selection (Nick Stuke)

In [34]:
sampler_cqm = LeapHybridCQMSampler()
sample_set_cqm=sampler_cqm.sample_cqm(cqm)

In [35]:
 n_samples = len(sample_set_cqm.record)
 feasible_samples = sample_set_cqm.filter(lambda d: d.is_feasible) 

In [52]:
best_feasible_cqm_solution=DWaveSampler.sample
if not feasible_samples: 
    raise Exception("No feasible solution could be found for this problem instance.")
else:
    best_feasible_cqm_solution = feasible_samples.first
    print(best_feasible_cqm_solution)

{"'Bremerhaven-Emden-137.0'": 0.0, "'Bremerhaven-Hamburg-50.0'": 1.0, "'Brunsbüttel-Bremerhaven-81.0'": 0.0, "'Brunsbüttel-Hamburg-36.0'": 0.0, "'Brunsbüttel-Kiel-54.0'": 0.0, "'Brunsbüttel-end-0.0'": 1.0, "'Emden-Bremerhaven-137.0'": 0.0, "'Hamburg-Bremerhaven-117.0'": 0.0, "'Hamburg-Brunsbüttel-36.0'": 1.0, "'Hamburg-Kiel-90.0'": 0.0, "'Kiel-Bremerhaven-135.0'": 0.0, "'start-Bremerhaven-0.0'": 1.0}


# Shortest Way with Dijkstra-Algorithm (Nick Stuke)

Dijkstra Algorithm

In [37]:
# define Graph with distances for every Point
graph={}
for index,port in df_ports.iterrows():
    routes=df_routes_withoutStardEnd[df_routes_withoutStardEnd['From']==port['From']]
    distances={}
    for i,r in routes.iterrows():
        distances[r['To']] = r['Distance']
    graph[port['From']] =distances
print(graph)


{'Bremerhaven': {'Emden': 137.0, 'Hamburg': 50.0}, 'Brunsbüttel': {'Bremerhaven': 81.0, 'Hamburg': 36.0, 'Kiel': 54.0}, 'Emden': {'Bremerhaven': 137.0}, 'Hamburg': {'Bremerhaven': 117.0, 'Brunsbüttel': 36.0, 'Kiel': 90.0}, 'Kiel': {'Bremerhaven': 135.0}}


In [38]:
import pprint
pp = pprint.PrettyPrinter(indent=4)

In [39]:
queue = [departure]
d = {node: {"shortest distance":float("inf"), "previous":None} for node in graph}
pp.pprint(d)
d[departure]["shortest distance"] = 0
while queue:
    print('Queue: '+str(queue))
    current = queue.pop(0)
    shortest_distance = d[current]["shortest distance"]
    for neighbour in graph[current]:

        dist_to_neighbour = graph[current][neighbour]

        if shortest_distance + dist_to_neighbour < d[neighbour]["shortest distance"]:

            d[neighbour] = {
                "shortest distance": shortest_distance + dist_to_neighbour,
                "previous": current
            }
            queue.append(neighbour)
pp.pprint(d)
print()

{   'Bremerhaven': {'previous': None, 'shortest distance': inf},
    'Brunsbüttel': {'previous': None, 'shortest distance': inf},
    'Emden': {'previous': None, 'shortest distance': inf},
    'Hamburg': {'previous': None, 'shortest distance': inf},
    'Kiel': {'previous': None, 'shortest distance': inf}}
Queue: ['Bremerhaven']
Queue: ['Emden', 'Hamburg']
Queue: ['Hamburg']
Queue: ['Brunsbüttel', 'Kiel']
Queue: ['Kiel']
{   'Bremerhaven': {'previous': None, 'shortest distance': 0},
    'Brunsbüttel': {'previous': 'Hamburg', 'shortest distance': 86.0},
    'Emden': {'previous': 'Bremerhaven', 'shortest distance': 137.0},
    'Hamburg': {'previous': 'Bremerhaven', 'shortest distance': 50.0},
    'Kiel': {'previous': 'Hamburg', 'shortest distance': 140.0}}



In [42]:
current = destination
print('Destination: '+str(destination))
print('Departure: '+str(departure))
dji_path = [current]
distance_sum=0.0
while current != departure:
    print(current)
    current = d[current]["previous"]
    dji_path.append(current)

dji_path = dji_path[::-1]
print('Distance: '+str(d[destination]["shortest distance"])+' Pfad: '+str(path))

Destination: Brunsbüttel
Departure: Bremerhaven
Brunsbüttel
Hamburg
Distance: 86.0 Pfad: ['Bremerhaven', 'Hamburg', 'Brunsbüttel']


# Result-Consideration

## Result-Homogenization (Nick Stuke)

In [75]:
result_csp=[departure]
result_cqm=[departure]
result_dji=dji_path

# homogenization of cqm-solution
solution=[]
for s in best_feasible_cqm_solution.sample:
    if (best_feasible_cqm_solution.sample[s]==1.0 and 
     s.find('end')==-1 and 
     s.find('start')==-1):
        solution.append(s)
currentP=departure
while currentP!=destination:
    for s in solution:
        s=s.replace("'","").split('-')
        if s[0].find(currentP)==0:
            currentP=s[1]
            result_cqm.append(s[1])
            break

# homogenization of csp-solution
solution=[]
for s in result.first.sample:
    if (result.first.sample[s]==1 and 
     s.find('end')==-1 and 
     s.find('start')==-1):
        solution.append(s)
currentP=departure
while currentP!=destination:
    for s in solution:
        s=s.split('-')
        if s[0].find(currentP)==0:
            currentP=s[1]
            result_csp.append(s[1])
            break


In [ ]:
print('result-cqm: '+str(result_cqm))
print('result-csp: '+str(result_csp))
print('result-dji: '+str(result_dji))